# 利用multiprocessing实现多进程
- multiprocessing是Python自带的用于管理进程的模块，通过合理地利用multiprocessing，我们可以充分榨干所使用机器的CPU运算性能


**multiprocessing需要将执行运算的语句放置于含有if name == 'main'：的脚本文件中下**

## Process
- Process是multiprocessing中最基础的类，用于创建进程

In [1]:
import multiprocessing
import datetime
import numpy as np
import os


def job():
    for j in range(100):
        _ = np.sum(np.random.rand(10000000))


process_list = []  # 首先初始化用于存放多个线程的列表process_list

for i in range(multiprocessing.cpu_count() - 1): # 用循环的方式创建了CPU核心数-1个进程并添加到process_list中
    process = multiprocessing.Process(target=job) # target: 需要执行的运算函数
    process_list.append(process)
        
# 要想实现真正的并行，需要现行对多个进程执行.start()，接着再对这些进程对象执行.join()，才能使得各个进程之间相互独立
        
for process in process_list:
    process.start()  # 在process_list创建完成之后，用循环的方式调用.start()方法将所有进程逐个激活

for process in process_list:
    process.join()  # .join()方法用于控制进程之间的并行

## Pool
- 除了上述的Process，在multiprocessing中还可以使用Pool来快捷地实现多进程

In [ ]:
from multiprocessing import Pool # 使用Pool这个类
import numpy as np
from pprint import pprint

def job(n):
    return np.mean(np.random.rand(n)), np.std(np.random.rand(n))

if __name__ == '__main__':
# Pool()中的.map()方法则根据前面传入的并行数量5，以多进程并行的方式执行
    with Pool(4) as p:
        pprint(p.map(job, [i**10 for i in range(1, 6)]))  # 将函数job利用.map()作用到后面传入序列每一个位置上

# 利用joblib实现多进程
- joblib将多进程的实现方式大大简化，使得我们可以在IPython交互式环境下中灵活地使用它
- scikit-learn中RandomForestClassifier等可以并行运算的算法都是通过joblib来实现的

## math函数示例

In [1]:
from joblib import Parallel, delayed  # 使用Joblib中的Parallel和delayed函数配置函数的并行
import numpy as np
from math import sqrt

Parallel(n_jobs=-1)(delayed(sqrt)(i ** 2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

## 并行打开网页

In [ ]:
# 另一个关于网页的例子

import urllib.request
from multiprocessing.dummy import Pool as ThreadPool

urls = [
    'http://www.python.org',
    'http://www.python.org/about/',
    'http://www.onlamp.com/pub/a/python/2003/04/17/metaclasses.html',
    'http://www.python.org/doc/',
    'http://www.python.org/download/',
    'http://www.python.org/getit/',
    'http://www.python.org/community/',
    'https://wiki.python.org/moin/',
    'http://planet.python.org/',
    'https://wiki.python.org/moin/LocalUserGroups',
    'http://www.python.org/psf/',
    'http://docs.python.org/devguide/',
    'http://www.python.org/community/awards/'
    # etc..
    ]

pool = ThreadPool(4)
results = pool.map(urllib.request.urlopen, urls)
#close the pool and wait for the work to finish
pool.close()
pool.join()

print(results)

## 单参数函数并行

In [5]:
# 自编函数示例（单参数）
import math

def my_fun(i):
    '''
    单参函数必须要可迭代
    '''
    return i**2

Parallel(n_jobs=-1)(delayed(my_fun)(i) for i in range(10))


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## 双（多）参数函数并行

In [7]:
import math

def my_fun_2p(i, j):
    '''
    双参函数必须要可迭代
    '''
    return i**j

Parallel(n_jobs=4)(delayed(my_fun_2p)(i, j) for i in range(10) for j in range(10))

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 8,
 16,
 32,
 64,
 128,
 256,
 512,
 1,
 3,
 9,
 27,
 81,
 243,
 729,
 2187,
 6561,
 19683,
 1,
 4,
 16,
 64,
 256,
 1024,
 4096,
 16384,
 65536,
 262144,
 1,
 5,
 25,
 125,
 625,
 3125,
 15625,
 78125,
 390625,
 1953125,
 1,
 6,
 36,
 216,
 1296,
 7776,
 46656,
 279936,
 1679616,
 10077696,
 1,
 7,
 49,
 343,
 2401,
 16807,
 117649,
 823543,
 5764801,
 40353607,
 1,
 8,
 64,
 512,
 4096,
 32768,
 262144,
 2097152,
 16777216,
 134217728,
 1,
 9,
 81,
 729,
 6561,
 59049,
 531441,
 4782969,
 43046721,
 387420489]